#### Notes

### Future notes
#### wanna be able to download the two files from RRCU, then run a py file that does the following
##### Cut the files, copy them into data folder 
##### take the old debit/credit files, put them into archive, rename them
##### rename new files

In [2]:
import polars as pl
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [3]:
df = pl.read_csv(r"C:\Users\Denne\OneDrive\Documents\GitHub\FinancialStreamline2\BankStreamline\Data\BankDataProd.csv", ignore_errors=True)


df = df.with_columns(
    pl.col("date").str.strptime(pl.Date, "%Y-%m-%d", strict=False)
)


# End date
days_backward = time.localtime().tm_mday
now = datetime.now()
EndOfLastMonth = now - timedelta(days=days_backward)
# Start date
LastYear = EndOfLastMonth - relativedelta(years=1)
StartDate = LastYear + relativedelta(months=1)
StartDate = StartDate.replace(day=1).date()
last_month_int = EndOfLastMonth.month


month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

df = df.sort("month")


df = df.filter(pl.col("date") >= pl.date(StartDate.year, StartDate.month, StartDate.day)) \
        .filter(pl.col("date") <= pl.date(EndOfLastMonth.year, EndOfLastMonth.month, EndOfLastMonth.day))

df.limit(3)

date,category,description,cost,controllable,monthName,month,day,year,weekDay,cardType,quarter
date,str,str,f64,i64,str,i64,i64,i64,str,str,i64
2025-01-31,"""food""","""POS 0131 0913 344858 CORPORATE…",1.75,1,"""Jan""",1,31,2025,"""Fri""","""Debit""",1
2025-01-31,"""gas""","""DC CHEVRON 0352098 EL DORADO A…",7.41,0,"""Jan""",1,31,2025,"""Fri""","""Debit""",1
2025-01-29,"""food""","""DC Subway COO61034 El Dorado A…",19.63,1,"""Jan""",1,29,2025,"""Wed""","""Debit""",1


### OG Graph Remake

In [4]:
# tempDF = df.group_by(["monthName", 'month']).agg(pl.sum("cost")) 

fig = px.bar(df.to_pandas(), x="monthName", y="cost", color="category", hover_data=['description', 'cardType', 'day'],
             title="Total Expense by Month",
             category_orders={"monthName": month_order},
             template='plotly_dark',
             labels={'monthName':''},
             text_auto=True,
             height=800,
             width=1600)

fig.show()
# fig.show()

### Smooth Graph Remake

In [5]:
tempDF = df.group_by(["monthName", 'month', 'category']).agg(pl.sum("cost")) 


fig = px.bar(tempDF.to_pandas(), x="monthName", y="cost", color="category",
             title="Total Expense by Month",
             category_orders={"monthName": month_order},
             template='plotly_dark',
             labels={'monthName':''},
             text_auto=True,
             height=800,
             width=1600)

fig.show()

### Only October Numbers (wip)

In [ ]:
tempDF = df.filter(pl.col('month') == last_month_int) \
            .group_by(['category']).agg(pl.sum("cost")) 

tempDF = tempDF.sort(by='cost', descending=True)

fig = px.bar(tempDF.to_pandas(), x="category", y="cost", color="category",
             title="Last Month Expenses by Category",
            #  category_orders={"monthName": month_order},
             template='plotly_dark',
             labels={'monthName':''},
             text_auto=True,
             height=800,
             width=1600)

fig.show()

### Last month against LTM average